In [28]:
import tensorflow as tf
import numpy as np
from SimDataDB import SimDataDB
%matplotlib inline
from matplotlib import pylab as plt
from matplotlib.pyplot import imshow

In [29]:
sdb = SimDataDB('pix.db')

In [6]:
sess = tf.InteractiveSession()

In [84]:
r=np.vstack(sdb.Query('select pic from static order by random() limit 10'))

Failed to sort.


In [86]:
r.shape

(10, 32, 32, 3)

Because the image data is stored as binary blobs, we can't use the SQLDataset in tensorflow. Instead, we make a generator that runs queries on the batch size. SimDataDB has the array-packing protocol implemented in it.

In [186]:
def gen():
    """Returns the 3 channel image"""
    while True:
        yield np.vstack(sdb.Query('select pic from static order by random() limit 10')),
data = tf.data.Dataset.from_generator(gen, (tf.float32,),(tf.TensorShape([None,32,32,3]),) )

In [187]:
def gen():
    """Returns the energy and the b&w fracture pic"""
    while True:
        r = sdb.Query('select W,pic from static order by random() limit 10')
        yield np.vstack([_[0] for _ in r]),1-np.stack([_[1][:,:,2] for _ in r]),
data = tf.data.Dataset.from_generator(gen, (tf.float32,tf.float32,),
                                      (tf.TensorShape([None,1]),tf.TensorShape([None,32,32,]),) )


In [177]:
#data = tf.data.experimental.SqlDataset("sqlite", 'pix.db',
#                                       'select W,G_c,pic from static',
#                                       (tf.float64,tf.float64,tf.Tensor))

In [178]:
iterator = data.make_one_shot_iterator()
nxt = iterator.get_next()

In [204]:
i_W = nxt[0]
i_pic = nxt[1]
A = tf.Variable(tf.truncated_normal(shape=(32,32,1),stddev=0.1))
o_W = tf.einsum('ijk,jkl->il',i_pic,A)
o_goal = tf.losses.mean_squared_error(o_W,i_W)

In [ ]:
o_goal = tf.reduce_mean

In [198]:
learning_rate = 1e-2
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(o_goal, var_list=[A])

In [208]:
init = tf.global_variables_initializer()

In [211]:
sess.run(init)
for i in range(100):
    sess.run(train_step)
    print sess.run(o_goal)

0.52785814
0.31802106
0.61531866
0.2239484
0.47308898
0.1452969
0.14818737
0.16958627
0.23533586
0.13696486
0.3546534
0.34861964
0.27850103
0.12332623
0.118767574
0.0887519
0.058497943
0.047551073
0.11004804
0.08665702
0.10844503
0.13586214
0.058221124
0.07092847
0.10384427
0.29018873
0.114653885
0.022364844
0.035875138
0.07467912
0.030135218
0.066757075
0.09339397
0.108720005
0.034771115
0.074159406
0.05923755
0.08981185
0.026017731
0.05595373
0.1444225
0.08853779
0.033817463
0.04324438
0.06227255
0.011674087
0.106775686
0.032865975
0.04674391
0.07925899
0.016697315
0.056715995
0.031872097
0.06184222
0.026263852
0.023658093
0.04021799
0.09865668
0.027315745
0.08453981
0.025653109
0.031095073
0.0253326
0.04846073
0.02523751
0.025190124
0.07690946
0.024904419
0.08268626
0.082659006
0.034884788
0.07868797
0.11244295
0.050963353
0.06300028
0.03421105
0.018123416
0.05442942
0.06271631
0.038230788
0.020121684
0.042839885
0.06845792
0.042734068
0.059472054
0.011089094
0.025263837
0.34942526


In [210]:
imshow(r[1][3,:,:])

IndexError: too many indices for array